# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
weather_df = pd.read_csv('../weatherpy/weatherAPI_data.csv',encoding = "ISO-8859-1")
weather_df.head()

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,1283285,Jumla,NP,29.2747,82.1838,04/30/21,47.55,80,60,3.98
1,4030556,Rikitea,PF,-23.1203,-134.9692,04/30/21,78.78,78,91,20.40
2,2381334,Atar,MR,20.5169,-13.0499,04/30/21,83.70,18,34,4.74
3,6167817,Torbay,CA,47.6666,-52.7314,04/30/21,38.10,87,90,13.80
4,2023469,Markova,RU,52.2132,104.2054,04/30/21,33.80,93,40,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Store latitude and longitude in locations
lat_lon = weather_df[['latitude', 'longitude']]

# Fill NaN values and convert to float
humid = weather_df["humidity"]
#lat_lon.head()

In [14]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.temp < 80) & (weather_df.temp > 70)
crit_windSpeed = weather_df.wind_speed < 15
crit_cloudiness = weather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,73,3469294,Bonito,BR,-21.1211,-56.4819,04/30/21,70.88,43,0,2.68
1,125,6323121,Florianópolis,BR,-27.6146,-48.5012,04/30/21,70.81,73,0,4.61
2,238,3981432,Todos Santos,MX,23.4500,-110.2167,04/30/21,70.52,79,0,4.00
3,245,3448903,São João da Barra,BR,-21.6403,-41.0511,04/30/21,73.40,68,0,6.91
4,275,3449711,Santo Amaro da Imperatriz,BR,-27.6881,-48.7786,04/30/21,70.03,73,0,4.61
5,291,107304,Buraidah,SA,26.3260,43.9750,04/30/21,75.20,33,0,4.61
6,295,2524815,Erice,IT,38.0370,12.5865,04/30/21,73.51,73,0,9.22
7,344,102527,Sakakah,SA,29.9697,40.2064,04/30/21,77.00,15,0,6.91
8,396,2065594,Mount Isa,AU,-20.7333,139.5000,04/30/21,71.60,53,0,4.61
9,435,2139521,Noumea,NC,-22.2763,166.4572,04/30/21,78.93,53,0,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed,Hotel Name
0,73,3469294,Bonito,BR,-21.1211,-56.4819,04/30/21,70.88,43,0,2.68,
1,125,6323121,Florianópolis,BR,-27.6146,-48.5012,04/30/21,70.81,73,0,4.61,
2,238,3981432,Todos Santos,MX,23.4500,-110.2167,04/30/21,70.52,79,0,4.00,
3,245,3448903,São João da Barra,BR,-21.6403,-41.0511,04/30/21,73.40,68,0,6.91,
4,275,3449711,Santo Amaro da Imperatriz,BR,-27.6881,-48.7786,04/30/21,70.03,73,0,4.61,


In [17]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

Missing field/result... skipping.


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [20]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…